In [68]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
from dateutil.relativedelta import relativedelta

In [69]:
polygon_api_key = "KkfCQ7fsZnx0yK4bhX9fD81QplTh0Pf3"
calendar = get_calendar("NYSE")

engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_2_database.db")

In [70]:
tickers = pd.read_sql("SELECT * FROM all_stocks", con = engine)["ticker"].values
stock_returns = pd.read_sql("SELECT * FROM stock_returns", con = engine)

In [71]:
momentum_dataset = pd.DataFrame(stock_returns['ticker'])

In [72]:
stock_returns = stock_returns.set_index('ticker')

In [73]:
gross_returns = stock_returns + 1
gross_returns.to_sql("gross_returns", con = engine, if_exists="replace")

In [98]:
#one thing that is a bit dodgy here is if there are market holidays in when any of the relative changes fall. Will look into using nyse.schedule() somehow
momentum_dataset['12m momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, (pd.to_datetime(gross_returns.columns[-1]).date() - relativedelta(months = 12)).strftime("%Y-%m-%d"):].cumprod(axis=1).iloc[:, -1]-1)

In [99]:
momentum_dataset['6m momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, (pd.to_datetime(gross_returns.columns[-1]).date() - relativedelta(months = 6)).strftime("%Y-%m-%d"):].cumprod(axis=1).iloc[:, -1]-1)

In [100]:
momentum_dataset['3m momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, (pd.to_datetime(gross_returns.columns[-1]).date() - relativedelta(months = 3)).strftime("%Y-%m-%d"):].cumprod(axis=1).iloc[:, -1]-1)

In [101]:
momentum_dataset['1m momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, (pd.to_datetime(gross_returns.columns[-1]).date() - relativedelta(months = 1)).strftime("%Y-%m-%d"):].cumprod(axis=1).iloc[:, -1]-1)

In [102]:
momentum_dataset['2w momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, (pd.to_datetime(gross_returns.columns[-1]).date() - relativedelta(weeks = 2)).strftime("%Y-%m-%d"):].cumprod(axis=1).iloc[:, -1]-1)

In [103]:
momentum_dataset['1w momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, (pd.to_datetime(gross_returns.columns[-1]).date() - relativedelta(weeks = 1)).strftime("%Y-%m-%d"):].cumprod(axis=1).iloc[:, -1]-1)

In [104]:
momentum_dataset['1d momentum'] = momentum_dataset['ticker'].map(gross_returns.loc[:, gross_returns.columns[-2]:].cumprod(axis=1).iloc[:, -1]-1)

In [105]:
momentum_dataset.to_sql("momentum_dataset", con = engine, if_exists="replace")

,ticker,12m momentum,6m momentum,3m momentum,1m momentum,2w momentum,1w momentum,1d momentum
0,NVDA,1.832837,0.647228,0.362507,0.047191,0.232447,0.052987,0.006770
1,TSLA,-0.055191,0.116053,0.223252,-0.124011,0.108027,0.028860,-0.013213
2,AAPL,0.279919,0.230352,0.188266,0.012859,0.063429,0.009434,0.001943
3,MSFT,0.292532,0.012486,-0.031892,-0.059037,0.035015,-0.010071,-0.017329
4,AMD,0.466780,-0.147806,-0.063678,-0.005710,0.136884,0.051710,-0.017933
...,...,...,...,...,...,...,...,...
495,APA,-0.331847,-0.057909,-0.055095,-0.089543,0.000703,-0.034261,0.014612
496,LITE,0.153187,0.180567,0.300725,0.067665,0.365251,0.132223,0.049300
497,BJ,0.251058,0.191964,0.028327,-0.092534,-0.017217,-0.032269,-0.055353
498,RF,0.238328,0.251234,0.170945,0.018758,0.095055,0.079508,0.046810
